<a href="https://colab.research.google.com/github/ronakdm/input-marginalization/blob/main/snli_figure2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
!pip install transformers
!git clone https://github.com/ronakdm/input-marginalization.git

In [2]:
%%bash
cd input-marginalization
git pull
cd ..

Already up to date.


In [11]:
from google.colab import drive
drive.mount('/content/gdrive',force_remount=True)
save_dir = "/content/gdrive/My Drive/NLP/imarg"

Mounted at /content/gdrive


In [7]:
%%capture
import sys
sys.path.append("input-marginalization")

from metrics import input_marginalization, colored_sentence
import torch
from transformers import BertForMaskedLM

In [8]:
%%capture
mlm = BertForMaskedLM.from_pretrained('bert-base-uncased').cuda()

In [12]:
#bert = torch.load(f"{save_dir}/bert_sst2.pt")
#cnn = torch.load(f"{save_dir}/cnn_sst2.pt")
lstm = torch.load(f"{save_dir}/lstm_sst2.pt")

In [13]:
lstm_sentences = [
  ("pacino is brilliant as the sleep - deprived dormer , his increasing weariness as much existential as it is physical .", 1),
  ("an important movie , a reminder of the power of file to move us and to make us examine our values .", 1),
  ("unflinchingly bleak and desperate", 0),
  ("i am sorry that i was unable to get the full brunt of this comedy .", 0),
]
bert_sentences = [
  ("it ' s a lovely film with lovely performances by buy and accorsi .", 1),
  ("more romantic , emotional and ultimately more satisfying than the teary - eyed original .", 1),
  ("it ' s a bit disappointing that it only manages to be decent instead of dead brilliant .", 0),
  ("suffers from the lack of a compelling or comprehensible narrative .", 0),   
]
cnn_sentences = [
  (". . . very funny , very enjoyable . . .", 1),
  ("lookin ' for sin , american - style ?", 0),
]

In [16]:
for sentence, label in lstm_sentences:
  att_scores = input_marginalization(lstm, sentence, mlm, target_label=label)
  print(att_scores)
  #colored_sentence(sentence, att_scores)

tensor([[ 0.1164,  0.4288, -0.7245, -0.9494, -0.5572, -0.9531, -0.9530, -0.9428,
         -0.9560, -0.9510, -0.9554, -0.2194, -0.9530, -0.9529, -0.7118, -0.9951,
         -1.0224, -0.9530, -0.9531, -0.9390, -0.8969, -0.9531, -0.9531, -0.9523,
         -0.9523, -0.9530, -0.8384]])
tensor([[-1.3153, -3.7263, -3.9799, -4.1224, -3.9934, -3.9925, -3.9900, -3.9947,
         -3.9933, -3.9925, -3.9934, -3.8094, -3.9940, -4.0075, -3.9818, -3.9934,
         -3.9942, -3.9972, -3.9674, -4.0296, -3.9933, -3.9933, -3.9940, -3.7421]])
tensor([[-0.1443, -1.3332, -2.4188, -2.4545, -2.4546, -2.3547, -2.0366, -2.4535,
         -2.3100, -3.0732]])
tensor([[-0.0021, -0.5803,  0.7145,  0.8137,  0.8092,  0.8129,  0.8124,  0.8302,
          0.8136,  0.8145,  0.8133,  0.8132,  0.5505,  1.2154,  0.8133,  0.8134,
          0.7773,  0.8133,  0.6590]])


In [ ]:
# try:
#   for sentence, label in bert_sentences:
#     att_scores = input_marginalization(bert, sentence, mlm, target_label=label)
#     color_sentence(sentence, att_scores)
# except KeyboardInterrupt:
#   print("Graceful Exit")

In [ ]:
# for sentence, label in cnn_sentences:
#   att_scores = input_marginalization(cnn, sentence, mlm, target_label=label)
#   color_sentence(sentence, att_scores)